In [2]:
import cv2
import mediapipe as mp
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.image
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.resnet50 import ResNet50 
from tensorflow.keras.applications import resnet50

In [3]:
new_model = tf.keras.models.load_model('saved_model/my_model/ResNet50V2')

In [4]:
new_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 2, 2, 2048)        23564800  
_________________________________________________________________
flatten_16 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 512)               4194816   
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 35)                17955     
Total params: 27,777,571
Trainable params: 4,212,771
Non-trainable params: 23,564,800
_________________________________________________________________


In [5]:
img_width, img_height = 64, 64
batch_size = 128
y_col = 'unknow'
x_col = 'path'


In [6]:
classes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
test_datagen = ImageDataGenerator(rescale = 1/255.0)
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (255, 0, 0)
thickness = 2

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      continue

    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    temp_image = np.full(image.shape, 255, dtype = np.uint8)

    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(  
            temp_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    
    
      
      matplotlib.image.imsave('image.png', temp_image)  
      img = cv2.imread('image.png')
      sample_mask_img = cv2.resize(img,(img_width,img_height))
      sample_mask_img = np.reshape(sample_mask_img,[1,img_width,img_height,3])
      sample_mask_img = sample_mask_img/255.0
      
      predictions = new_model.predict(sample_mask_img)
      
      label = classes[np.argmax(predictions[0])]
    
      image = cv2.putText(image, f'Letter is : {label}', org, font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow('Hands Recognitiom', image)
    
    
    if cv2.waitKey(1) & 0xFF == 27:
      break
cap.release()